https://medium.com/datadriveninvestor/time-series-prediction-using-sarimax-a6604f258c56

In [1]:
from platform import python_version
print(python_version())

3.11.9


In [2]:
%pwd

'/Users/joseangelvelasco/PycharmProjects/notebooks'

In [3]:
# %pip install pmdarima

In [4]:
#Importing required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
from pandas.tseries.offsets import BDay
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import pmdarima as pm



ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
register_matplotlib_converters()
%matplotlib inline

In [ ]:
dataset = pd.read_csv("data/aapl.csv")

In [ ]:
dataset.head(2)

In [ ]:
dataset.tail(2)

In [ ]:
dataset['Mean'] = (dataset['Low'] + dataset['High'])/2
dataset.head(2)

In [ ]:
dataset.info()

In [ ]:
dataset.head(2)

In [ ]:
# preparing the dataset by shoifting open , close, Low, High  by 1
steps=-1
dataset_for_prediction= dataset.copy()
dataset_for_prediction['Actual']=dataset_for_prediction['Mean'].shift(steps)
dataset_for_prediction.head(3)

In [ ]:
dataset_for_prediction=dataset_for_prediction.dropna()

In [ ]:
dataset_for_prediction.head(3)

In [ ]:
dataset_for_prediction[2629:2640]

In [ ]:
#dataset_for_prediction['Date']=dataset_for_prediction['Date'].asfreq(BDay())

In [ ]:
#dataset_for_prediction.to_csv('Check data.csv')

In [ ]:
dataset_for_prediction['Date'] =pd.to_datetime(dataset_for_prediction['Date'])
dataset_for_prediction.index= dataset_for_prediction['Date']
#dataset_test.index.freq='B'

In [ ]:
dataset_for_prediction.head(2)

In [ ]:
dataset_for_prediction.info()

In [ ]:
dataset_for_prediction['Mean'].plot(color='green', figsize=(15,2))
plt.legend([  'Mean'])
plt.title(" Apple Stock Value")

In [ ]:
dataset_for_prediction['Volume'].plot(color='blue', figsize=(15,2))
plt.title(" Apple  Stock Volume")

In [ ]:
# normalizing input features
sc_in = MinMaxScaler(feature_range=(0, 1))
scaled_input = sc_in.fit_transform(dataset_for_prediction[['Low', 'High','Open', 'Close', 'Volume', 'Adj Close', 'Mean']])
scaled_input = pd.DataFrame(scaled_input)
X = scaled_input

In [ ]:
sc_out = MinMaxScaler(feature_range=(0, 1))
scaler_output = sc_out.fit_transform(dataset_for_prediction[['Actual']])
scaler_output = pd.DataFrame(scaler_output)
y = scaler_output

In [ ]:
X = scaled_input

In [ ]:
X.rename(columns={0:'Low', 1:'High', 2:'Open', 3:'Close', 4:'Volume', 5:'Adj Close', 6:'Mean'}, inplace=True)
X = pd.DataFrame(X)
X.index = dataset_for_prediction.index
X.tail(2)

In [ ]:
y.rename(columns={0:'Stock Price next day'}, inplace= True)
y.index=dataset_for_prediction.index
y.head(2)

In [ ]:
seas_d = sm.tsa.seasonal_decompose(X['Mean'],model='add',freq=365);
fig=seas_d.plot()
fig.set_figheight(4)
plt.show()

In [ ]:
train_size=int(len(dataset) *0.80)
test_size = int(len(dataset)) - train_size
print(test_size)

In [ ]:
train_X, train_y = X[:train_size].dropna(), y[:train_size].dropna()
test_X, test_y = X[train_size:].dropna(), y[train_size:].dropna()

In [ ]:
test_X.columns

In [ ]:
y_test=y['Stock Price next day'][:train_size].dropna()

In [ ]:
type(y_test)

In [ ]:
def test_adf(series, title=''):
    dfout={}
    dftest=sm.tsa.adfuller(series.dropna(), autolag='AIC', regression='ct')
    for key,val in dftest[4].items():
        dfout[f'critical value ({key})']=val
    if dftest[1]<=0.05:
        print("Strong evidence against Null Hypothesis")
        print("Reject Null Hypothesis - Data is Stationary")
        print("Data is Stationary", title)
    else:
        print("Strong evidence for Null Hypothesis")
        print("Accept Null Hypothesis - Data is not Stationary")
        print("(!) Data is NOT Stationary for:", title)

In [ ]:
test_adf(y_test, "Stock Price")

In [ ]:
test_adf(y_test.diff(), "Stock Price")

In [ ]:
#Correlograms
fig,ax= plt.subplots(2,1, figsize=(10,5))
fig=sm.tsa.graphics.plot_acf(y_test, lags=50, ax=ax[0])
fig=sm.tsa.graphics.plot_pacf(y_test, lags=50, ax=ax[1])
plt.show()

In [ ]:
from pyramid.datasets import load_wineind

In [ ]:
step_wise = pm.arima.auto_arima(train_y, 
                     exogenous = train_X,
                     start_p=1, 
                     start_q=1, 
                     max_p=7, 
                     max_q=7, 
                     d=1,
                     max_d=7,
                     trace=True, 
                     error_action='ignore', 
                     suppress_warnings=True, 
                     stepwise=True)

In [ ]:
step_wise.summary()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
model= SARIMAX(train_y, 
           exog=train_X,
              order=(0,1,1),
              enforce_invertibility=False, enforce_stationarity=False)

In [ ]:
results= model.fit()

In [ ]:
test_X

In [ ]:
test_X.info()

In [ ]:
test_X[['Low', 'High', 'Open', 'Close', 'Volume', 'Adj Close', 'Mean']]

In [ ]:
train_X

In [ ]:
step_wise=auto_arima(train_y, 
                     start_p=1,  start_q=1, 
                     max_p=7,  max_q=7, 
                     d=1, max_d=7,
                     trace=True, 
                     error_action='ignore', 
                     suppress_warnings=True, 
                     stepwise=True)

In [ ]:
step_wise.summary()

In [ ]:
model= SARIMAX(train_y,            
              order=(2,1,0),
              enforce_invertibility=False, enforce_stationarity=False)
results = model.fit()
forecast = results.forecast(steps=2)

In [ ]:
a = train_size+test_size-(steps)-train_size
steps

In [ ]:
model= SARIMAX(train_y, 
               exog=train_X,            
              order=(0,1,1),
              enforce_invertibility=False, enforce_stationarity=False)
results = model.fit()
predictions= results.predict(start =train_size, end=train_size+test_size+(steps)-1,exog=test_X)
print(len(predictions))

In [ ]:
forecast_1= results.forecast(steps=test_size-1, exog=test_X)

In [ ]:
act= pd.DataFrame(scaler_output.iloc[train_size:, 0])


In [ ]:
forecast_apple= pd.DataFrame(forecast_1)
forecast_apple.reset_index(drop=True, inplace=True)
forecast_apple.index=test_X.index
forecast_apple['Actual'] =scaler_output.iloc[train_size:, 0]
forecast_apple.rename(columns={0:'Forecast'}, inplace=True)

In [ ]:
forecast_apple['Forecast'].plot(legend=True)
forecast_apple['Actual'].plot(legend=True)

In [ ]:
predictions=pd.DataFrame(predictions)
predictions.reset_index(drop=True, inplace=True)
predictions.index=test_X.index
predictions['Actual'] = act['Stock Price next day']
predictions.rename(columns={0:'Pred'}, inplace=True)



In [ ]:
predictions['Actual'].plot(figsize=(20,8), legend=True, color='blue')
predictions['Pred'].plot(legend=True, color='red', figsize=(20,8))



In [ ]:
from statsmodels.tools.eval_measures import rmse
error=rmse(predictions['Pred'], predictions['Actual'])
error

## Inverse Scalar

In [ ]:
X.head(2)


In [ ]:
# invert predictions, scale values back to real index/price range.
testPredict = sc_out.inverse_transform(predictions[['Pred']])
testActual = sc_out.inverse_transform(predictions[['Actual']])

In [ ]:
act_1=pd.DataFrame(testActual)

In [ ]:
#act_pred=pd.DataFrame()
act_pred= pd.DataFrame(testPredict)
act_pred.reset_index(drop=True, inplace=True)
act_pred.index=test_X.index
act_pred['Actual'] = act['Stock Price next day']
predictions.rename(columns={0:'Pred'}, inplace=True)

In [ ]:
y.head(2)

In [ ]:
train_X, train_y = X.iloc[:train_size,[4,5,6]].dropna(), y[:train_size].dropna()
test_X, test_y = X.iloc[train_size:,[4,5,6]], y[train_size:]

In [ ]:

train_X.shape
train_y.shape

In [ ]:
from pmdarima.arima import auto_arima
step_wise=auto_arima(train_y, 
                     exogenous= train_X,
                     start_p=1,  start_q=1, 
                     max_p=7,  max_q=7, 
                     d=1, max_d=7,
                     trace=True, 
                     error_action='ignore', 
                     suppress_warnings=True, 
                     stepwise=True)

In [ ]:
step_wise.summary()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
model= SARIMAX(train_y, 
           exog=train_X,
              order=(1,1,1),
              enforce_invertibility=False, enforce_stationarity=False)

In [ ]:
results=model.fit()

In [ ]:
predictions= results.predict(start =train_size, end=train_size+829, 
                             exog=test_X)

In [ ]:
predictions=pd.DataFrame(predictions)
predictions.reset_index(drop=True, inplace=True)
test=pd.DataFrame(dataset.iloc[train_size:, 0])
test.reset_index(drop=True, inplace=True)
act= pd.DataFrame(scaler_output.iloc[train_size:, 0])
predictions['Date'] = test['Date']
predictions.rename(columns={0:'pred', 1:'Date'}, inplace=True)
predictions.index= predictions['Date']
predictions['Actual'] = act['Stock Price next day']


In [ ]:
predictions['Actual'].plot(figsize=(20,8), legend=True, color='blue')
#y['2008'].plot(figsize=(20,8), legend=True, color='green')
predictions['pred'].plot(legend=True, color='red', figsize=(20,8))

In [ ]:
# invert predictions, scale values back to real index/price range.
#scaler_output
#trainPredict = sc_out.inverse_transform(predictions['pred'])
#testPredict = sc_in.inverse_transform(predictions['Actual'])

In [ ]:
import xgboost as xgb
from xgboost import plot_importance, plot_tree

In [ ]:
from sklearn.model_selection import GridSearchCV
xgb1 = xgb.XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [100,500, 1000]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 5,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(train_X,
         train_y)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

In [ ]:
reg_1 = xgb.XGBRegressor(n_estimators=1500, learning_rate=0.05, max_depth=5,subsample=0.7)
reg_1.fit(train_X, train_y)
_ = plot_importance(reg_1, height=0.9)

In [ ]:
pred=reg_1.predict(test_X)

In [ ]:
type(pred)

In [ ]:
predictions=pd.DataFrame(pred)
predictions.reset_index(drop=True, inplace=True)
test=pd.DataFrame(dataset.iloc[train_size:, 0])
test.reset_index(drop=True, inplace=True)
act= pd.DataFrame(scaler_output.iloc[train_size:, 0])
predictions['Date'] = test['Date']
predictions.rename(columns={0:'pred', 1:'Date'}, inplace=True)
predictions.index= predictions['Date']
predictions['Actual'] = act['Stock Price next day']


In [ ]:
predictions['Actual'].plot(figsize=(20,8), legend=True, color='blue')
#y['2008'].plot(figsize=(20,8), legend=True, color='green')
predictions['pred'].plot(legend=True, color='red', figsize=(20,8))

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator
from keras import Sequential
from keras.layers import Dense, LSTM, BatchNormalization, Dropout,Activation
n_input=3
n_features =3
from sklearn.preprocessing import StandardScaler

In [ ]:
sc_tinput=StandardScaler()
sc_toutput=StandardScaler()

In [ ]:
sc_X = sc_tinput.fit_transform(dataset_for_prediction[[ 'Volume', 'Adj Close', 'Mean']])
sc_Y = sc_toutput.fit_transform(dataset_for_prediction[['Actual']])

In [ ]:
sc_train_X=sc_X[:train_size]
sc_test_X=sc_X[train_size:]
sc_train_y=sc_Y[:train_size]
sc_test_y=sc_Y[train_size:]

In [ ]:
sc_test_y

In [ ]:
generator_train= TimeseriesGenerator(sc_train_X, sc_train_y, length=n_input, batch_size=32)
generator_test= TimeseriesGenerator(sc_test_X, sc_test_y, length=n_input, batch_size=32)


In [ ]:
model = Sequential()
model.add(LSTM(units=150, return_sequences= True, input_shape=(n_input, n_features)))
model.add(Dropout(0.2))
model.add(BatchNormalization())    
model.add(Activation('relu'))
model.add(LSTM(units=150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=30, return_sequences=True))
model.add(LSTM(units=30))
model.add(Dense(1))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit_generator(generator_train, epochs=200)

In [ ]:
myloss=model.history.history['loss']

In [ ]:
plt.plot(range(len(myloss)), myloss)

In [ ]:
predict_train= model.predict_generator(generator_train)
predict_test= model.predict_generator(generator_test)

In [ ]:
sc_test_y

In [ ]:
predictions=pd.DataFrame(predict_test)
predictions.reset_index(drop=True, inplace=True)
test=pd.DataFrame(dataset.iloc[train_size:, 0])
test.reset_index(drop=True, inplace=True)
act= pd.DataFrame(sc_test_y)
act.rename(columns={0:'Stock Price next day'}, inplace=True)
predictions['Actual'] = act['Stock Price next day']
predictions['Date'] = test['Date']
predictions.rename(columns={0:'pred', 1:'Date'}, inplace=True)
predictions.index= predictions['Date']



In [ ]:
predictions

In [ ]:
predictions['Actual'].plot(figsize=(20,8), legend=True, color='blue')
#y['2008'].plot(figsize=(20,8), legend=True, color='green')
predictions['pred'].plot(legend=True, color='red', figsize=(20,8))

In [ ]:
train_X

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA
model= ARIMA(train_y, 
           exog=train_X,
              order=(0,1,1))

In [ ]:
results= model.fit()

In [ ]:
predictions= results.predict(start =train_size, end=train_size+829, 
                             exog=test_X)

In [ ]:
predictions=pd.DataFrame(predictions)
predictions.reset_index(drop=True, inplace=True)
test=pd.DataFrame(dataset.iloc[train_size:, 0])
test.reset_index(drop=True, inplace=True)
act= pd.DataFrame(scaler_output.iloc[train_size:, 0])
predictions['Date'] = test['Date']
predictions.rename(columns={0:'pred', 1:'Date'}, inplace=True)
predictions.index= predictions['Date']
predictions['Actual'] = act['Stock Price next day']


In [ ]:
predictions['Actual'].plot(figsize=(20,8), legend=True, color='blue')
#y['2008'].plot(figsize=(20,8), legend=True, color='green')
predictions['pred'].plot(legend=True, color='red', figsize=(20,8))